# META 2 - Ex 4.5 e 4.6

In [1]:
import pandas as pd
import numpy as np
from ReliefF import ReliefF
#from skfeature.function.similarity_based import fisher_score      # pip install skfeature-chappers
pd.set_option('display.max_columns', None)  # Apenas para mostrar todas as colunas dos DataFrames

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile

from sklearn.preprocessing import StandardScaler
# https://www.datacamp.com/community/tutorials/feature-selection-python

## Carrega os dados das features calculadas na meta 1

In [2]:
all_features = pd.read_csv("allFeatures.csv").drop(columns=["ACC_X_Zero_Crossing_Rate",
                                                            "ACC_Y_Zero_Crossing_Rate",
                                                            "ACC_Z_Zero_Crossing_Rate",
                                                            "GYRO_X_Zero_Crossing_Rate",
                                                            "GYRO_Y_Zero_Crossing_Rate",
                                                            "GYRO_Z_Zero_Crossing_Rate",
                                                            "MAG_X_Zero_Crossing_Rate",
                                                            "MAG_Y_Zero_Crossing_Rate",
                                                            "MAG_Z_Zero_Crossing_Rate",
                                                           ])
all_features

,ACC_X_mean,ACC_X_std,ACC_X_median,ACC_X_skew,ACC_X_kurtosis,ACC_X_IQR,ACC_X_calculate_entropy,ACC_X_spectral_entropy,ACC_X_ENERGY,ACC_X_DF,ACC_X_averaged_Deviation,ACC_Y_mean,ACC_Y_std,ACC_Y_median,ACC_Y_skew,ACC_Y_kurtosis,ACC_Y_IQR,ACC_Y_calculate_entropy,ACC_Y_spectral_entropy,ACC_Y_ENERGY,ACC_Y_DF,ACC_Y_averaged_Deviation,ACC_Z_mean,ACC_Z_std,ACC_Z_median,ACC_Z_skew,ACC_Z_kurtosis,ACC_Z_IQR,ACC_Z_calculate_entropy,ACC_Z_spectral_entropy,ACC_Z_ENERGY,ACC_Z_DF,ACC_Z_averaged_Deviation,GYRO_X_mean,GYRO_X_std,GYRO_X_median,GYRO_X_skew,GYRO_X_kurtosis,GYRO_X_IQR,GYRO_X_calculate_entropy,GYRO_X_spectral_entropy,GYRO_X_ENERGY,GYRO_X_DF,GYRO_X_averaged_Deviation,GYRO_Y_mean,GYRO_Y_std,GYRO_Y_median,GYRO_Y_skew,GYRO_Y_kurtosis,GYRO_Y_IQR,GYRO_Y_calculate_entropy,GYRO_Y_spectral_entropy,GYRO_Y_ENERGY,GYRO_Y_DF,GYRO_Y_averaged_Deviation,GYRO_Z_mean,GYRO_Z_std,GYRO_Z_median,GYRO_Z_skew,GYRO_Z_kurtosis,GYRO_Z_IQR,GYRO_Z_calculate_entropy,GYRO_Z_spectral_entropy,GYRO_Z_ENERGY,GYRO_Z_DF,GYRO_Z_averaged_Deviation,MAG_X_mean,MAG_X_std,MAG_X_median,MAG_X_skew,MAG_X_kurtosis,MAG_X_IQR,MAG_X_calculate_entropy,MAG_X_spectral_entropy,MAG_X_ENERGY,MAG_X_DF,MAG_X_averaged_Deviation,MAG_Y_mean,MAG_Y_std,MAG_Y_median,MAG_Y_skew,MAG_Y_kurtosis,MAG_Y_IQR,MAG_Y_calculate_entropy,MAG_Y_spectral_entropy,MAG_Y_ENERGY,MAG_Y_DF,MAG_Y_averaged_Deviation,MAG_Z_mean,MAG_Z_std,MAG_Z_median,MAG_Z_skew,MAG_Z_kurtosis,MAG_Z_IQR,MAG_Z_calculate_entropy,MAG_Z_spectral_entropy,MAG_Z_ENERGY,MAG_Z_DF,MAG_Z_averaged_Deviation,ACC_AI,ACC_SMA,GYRO_AI,GYRO_SMA,MAG_AI,MAG_SMA,AAE,ARE,AVH,AVG,CAGH,EVA_heading,EVA_gravity,Atividade
0,2.329929,0.253342,2.32045,0.864882,15.348457,0.057700,5.984788,4.906173,3.573913e-08,8.031373,0.229741,9.552531,0.205244,9.56030,-1.156770,15.456326,0.036425,6.262872,5.225666,1.078168e-08,15.560784,-0.305426,1.915877,0.094768,1.90960,0.892216,17.071350,0.056150,6.343898,3.894447,2.791592e-09,0.501961,-0.238967,0.069727,1.777248,-0.291390,3.040172,9.843500,0.456001,6.672425,3.534973,0.000847,0.000000,4.011261,1.236437,4.711550,0.373760,2.380225,10.204243,1.029751,6.351299,4.223086,0.016295,0.501961,6.434071,0.917743,3.406595,0.240570,3.308385,13.810821,1.541077,6.672425,3.693402,1.549349e-02,0.501961,4.122310,0.715363,0.008683,0.716360,-0.244460,-0.514508,0.012430,3.875942,4.429686,1.851735e-11,18.572549,0.001049,0.915986,0.016389,0.916670,-0.394517,-0.198916,0.025800,4.688015,4.820018,1.881234e-10,10.541176,-0.005029,0.968439,0.007947,0.969300,0.176119,1.003543,0.010960,3.708418,4.986885,6.291531e-12,0.501961,0.010002,10.022046,13.798338,3.035379,4.597686,1.512894,2.599787,1.643747e-08,0.010879,9.612946,1.892059,-0.073102,0.063553,0.008893,Stand
1,1.995713,0.492077,2.11045,0.352800,0.206786,0.781450,6.441937,4.604139,2.475833e-04,0.501961,-0.568269,9.622844,0.289979,9.58640,-0.622660,3.804620,0.252300,6.527770,5.224757,2.955696e-05,14.556863,-0.061795,1.850789,0.154012,1.88660,0.218335,1.777674,0.210625,6.586593,4.305345,4.431324e-06,0.501961,-0.024485,0.938353,4.391607,-0.214645,2.574104,8.615281,2.084835,6.672425,3.976708,6.296641,1.003922,-0.048711,4.030593,7.478996,1.258600,1.474831,3.117834,5.331845,6.574386,4.403701,34.088568,1.003922,-0.085070,-2.193965,7.878282,-0.814350,-0.316066,1.182581,5.408017,6.672425,4.347456,4.369014e+01,1.003922,-2.037801,0.678481,0.042116,0.701860,-0.422282,-1.514361,0.078680,5.142158,1.922901,2.133464e-08,0.501961,-0.058774,0.944406,0.037944,0.934520,0.337875,-1.008749,0.063490,5.547247,3.490057,1.042216e-08,0.501961,0.045274,1.005333,0.038724,0.993420,0.305434,-1.459463,0.070200,5.166477,1.421156,1.237417e-08,0.501961,0.055605,10.014860,13.469346,8.652250,13.019811,1.538384,2.628220,9.385720e-05,28.025117,9.666724,1.825292,-0.148854,0.239766,0.023487,Stand
2,1.413991,0.352778,1.32980,0.886113,2.451716,0.357900,6.613602,3.829667,1.597482e-05,0.501961,-0.535876,9.742375,0.229803,9.73160,-0.525898,2.210588,0.230700,6.652817,4.458313,4.021650e-06,10.039216,-0.230806,1.781629,0.099065,1.78425,0.153507,1.338147,0

In [3]:
# Separa os dados em 2 DataFrames
# target: Coluna das atividades
# all_features_without_target: Os valores das várias features

target = pd.DataFrame(all_features["Atividade"])
all_features_without_target = all_features.drop(columns=["Atividade"])
display(target)
display(all_features_without_target)

,Atividade
0,Stand
1,Stand
2,Stand
3,Stand
4,Stand
...,...
14464,Climb Stairs and Talk to Walk and Talk
14465,Climb Stairs and Talk to Walk and Talk
14466,Climb Stairs and Talk to Walk and Talk
14467,Climb Stairs and Talk to Walk and Talk


,ACC_X_mean,ACC_X_std,ACC_X_median,ACC_X_skew,ACC_X_kurtosis,ACC_X_IQR,ACC_X_calculate_entropy,ACC_X_spectral_entropy,ACC_X_ENERGY,ACC_X_DF,ACC_X_averaged_Deviation,ACC_Y_mean,ACC_Y_std,ACC_Y_median,ACC_Y_skew,ACC_Y_kurtosis,ACC_Y_IQR,ACC_Y_calculate_entropy,ACC_Y_spectral_entropy,ACC_Y_ENERGY,ACC_Y_DF,ACC_Y_averaged_Deviation,ACC_Z_mean,ACC_Z_std,ACC_Z_median,ACC_Z_skew,ACC_Z_kurtosis,ACC_Z_IQR,ACC_Z_calculate_entropy,ACC_Z_spectral_entropy,ACC_Z_ENERGY,ACC_Z_DF,ACC_Z_averaged_Deviation,GYRO_X_mean,GYRO_X_std,GYRO_X_median,GYRO_X_skew,GYRO_X_kurtosis,GYRO_X_IQR,GYRO_X_calculate_entropy,GYRO_X_spectral_entropy,GYRO_X_ENERGY,GYRO_X_DF,GYRO_X_averaged_Deviation,GYRO_Y_mean,GYRO_Y_std,GYRO_Y_median,GYRO_Y_skew,GYRO_Y_kurtosis,GYRO_Y_IQR,GYRO_Y_calculate_entropy,GYRO_Y_spectral_entropy,GYRO_Y_ENERGY,GYRO_Y_DF,GYRO_Y_averaged_Deviation,GYRO_Z_mean,GYRO_Z_std,GYRO_Z_median,GYRO_Z_skew,GYRO_Z_kurtosis,GYRO_Z_IQR,GYRO_Z_calculate_entropy,GYRO_Z_spectral_entropy,GYRO_Z_ENERGY,GYRO_Z_DF,GYRO_Z_averaged_Deviation,MAG_X_mean,MAG_X_std,MAG_X_median,MAG_X_skew,MAG_X_kurtosis,MAG_X_IQR,MAG_X_calculate_entropy,MAG_X_spectral_entropy,MAG_X_ENERGY,MAG_X_DF,MAG_X_averaged_Deviation,MAG_Y_mean,MAG_Y_std,MAG_Y_median,MAG_Y_skew,MAG_Y_kurtosis,MAG_Y_IQR,MAG_Y_calculate_entropy,MAG_Y_spectral_entropy,MAG_Y_ENERGY,MAG_Y_DF,MAG_Y_averaged_Deviation,MAG_Z_mean,MAG_Z_std,MAG_Z_median,MAG_Z_skew,MAG_Z_kurtosis,MAG_Z_IQR,MAG_Z_calculate_entropy,MAG_Z_spectral_entropy,MAG_Z_ENERGY,MAG_Z_DF,MAG_Z_averaged_Deviation,ACC_AI,ACC_SMA,GYRO_AI,GYRO_SMA,MAG_AI,MAG_SMA,AAE,ARE,AVH,AVG,CAGH,EVA_heading,EVA_gravity
0,2.329929,0.253342,2.32045,0.864882,15.348457,0.057700,5.984788,4.906173,3.573913e-08,8.031373,0.229741,9.552531,0.205244,9.56030,-1.156770,15.456326,0.036425,6.262872,5.225666,1.078168e-08,15.560784,-0.305426,1.915877,0.094768,1.90960,0.892216,17.071350,0.056150,6.343898,3.894447,2.791592e-09,0.501961,-0.238967,0.069727,1.777248,-0.291390,3.040172,9.843500,0.456001,6.672425,3.534973,0.000847,0.000000,4.011261,1.236437,4.711550,0.373760,2.380225,10.204243,1.029751,6.351299,4.223086,0.016295,0.501961,6.434071,0.917743,3.406595,0.240570,3.308385,13.810821,1.541077,6.672425,3.693402,1.549349e-02,0.501961,4.122310,0.715363,0.008683,0.716360,-0.244460,-0.514508,0.012430,3.875942,4.429686,1.851735e-11,18.572549,0.001049,0.915986,0.016389,0.916670,-0.394517,-0.198916,0.025800,4.688015,4.820018,1.881234e-10,10.541176,-0.005029,0.968439,0.007947,0.969300,0.176119,1.003543,0.010960,3.708418,4.986885,6.291531e-12,0.501961,0.010002,10.022046,13.798338,3.035379,4.597686,1.512894,2.599787,1.643747e-08,0.010879,9.612946,1.892059,-0.073102,0.063553,0.008893
1,1.995713,0.492077,2.11045,0.352800,0.206786,0.781450,6.441937,4.604139,2.475833e-04,0.501961,-0.568269,9.622844,0.289979,9.58640,-0.622660,3.804620,0.252300,6.527770,5.224757,2.955696e-05,14.556863,-0.061795,1.850789,0.154012,1.88660,0.218335,1.777674,0.210625,6.586593,4.305345,4.431324e-06,0.501961,-0.024485,0.938353,4.391607,-0.214645,2.574104,8.615281,2.084835,6.672425,3.976708,6.296641,1.003922,-0.048711,4.030593,7.478996,1.258600,1.474831,3.117834,5.331845,6.574386,4.403701,34.088568,1.003922,-0.085070,-2.193965,7.878282,-0.814350,-0.316066,1.182581,5.408017,6.672425,4.347456,4.369014e+01,1.003922,-2.037801,0.678481,0.042116,0.701860,-0.422282,-1.514361,0.078680,5.142158,1.922901,2.133464e-08,0.501961,-0.058774,0.944406,0.037944,0.934520,0.337875,-1.008749,0.063490,5.547247,3.490057,1.042216e-08,0.501961,0.045274,1.005333,0.038724,0.993420,0.305434,-1.459463,0.070200,5.166477,1.421156,1.237417e-08,0.501961,0.055605,10.014860,13.469346,8.652250,13.019811,1.538384,2.628220,9.385720e-05,28.025117,9.666724,1.825292,-0.148854,0.239766,0.023487
2,1.413991,0.352778,1.32980,0.886113,2.451716,0.357900,6.613602,3.829667,1.597482e-05,0.501961,-0.535876,9.742375,0.229803,9.73160,-0.525898,2.210588,0.230700,6.652817,4.458313,4.021650e-06,10.039216,-0.230806,1.781629,0.099065,1.78425,0.153507,1.338147,0.099075,6.652817,4.548

## 4.5 - ReliefF

In [4]:
n_features_to_keep = 10

fs = ReliefF(n_neighbors=10, n_features_to_keep=n_features_to_keep)

X_train = fs.fit_transform(all_features_without_target.to_numpy(), target.to_numpy().reshape(1, -1)[0])
X_train

array([[ 1.24300000e-02,  1.09600000e-02,  9.69300000e-01, ...,
         8.03137255e+00,  5.01960784e-01,  3.73760000e-01],
       [ 7.86800000e-02,  7.02000000e-02,  9.93420000e-01, ...,
         5.01960784e-01,  1.00392157e+00,  1.25860000e+00],
       [ 3.46825000e-02,  3.67500000e-02,  1.06360000e+00, ...,
         5.01960784e-01,  3.51372549e+00,  1.83065000e+00],
       ...,
       [ 1.57867500e-01,  1.62287500e-01,  5.51535000e-01, ...,
         5.01960784e-01,  4.51764706e+00, -1.05263900e+00],
       [ 2.62422500e-01,  1.38150000e-01,  4.90135000e-01, ...,
         5.01960784e-01,  1.00392157e+00,  3.86730000e+00],
       [ 2.03932500e-01,  1.14037500e-01,  4.97810000e-01, ...,
         2.00784314e+00,  2.50980392e+00, -1.64380000e+01]])

In [5]:
def reliefF(data,number_features):
    
    activities_labels = ["", "Stand", "Sit", "Sit and Talk", "Walk", "Walk and Talk", "Climb Stair", "Climb Stair and Talk", "Stand to Sit", "Sit to Stand", "Stand to Sit and Talk", "Sit to Stand and Talk", "Stand to Walk", "Walk to Stand", "Stand to Climb Stairs / Stand to Climb Stairs and Talk", "Climb Stairs to Walk", "Climb Stairs and Talk to Walk and Talk"]

    X = data.drop(columns = ["Atividade"]).to_numpy()
    y = np.array(list(map(lambda x: activities_labels.index(x),data["Atividade"])))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    fs = ReliefF(n_neighbors=100, n_features_to_keep=number_features)
    X_train = fs.fit_transform(X_train, y_train)
    X_test_subset = fs.transform(X_test)
    if X_test_subset.shape[1] == number_features:
        print("Sucess!")
    else:
        print("Nop!")
        
    return list(map(lambda i: list(all_features.columns)[i],fs.top_features))[:number_features]

In [6]:
reliefF(all_features,10)

Sucess!


['MAG_X_IQR',
 'MAG_Z_IQR',
 'MAG_Y_IQR',
 'MAG_Z_median',
 'MAG_X_median',
 'MAG_Y_median',
 'GYRO_Y_median',
 'ACC_X_IQR',
 'ACC_X_median',
 'ACC_Z_median']

## 4.5 - Fisher Score - Tiago

## 4.5 - Fisher Score - Miguel

In [7]:
def fisher_score(data,number_features):
    activities_labels = ["", "Stand", "Sit", "Sit and Talk", "Walk", "Walk and Talk", "Climb Stair", "Climb Stair and Talk", "Stand to Sit", "Sit to Stand", "Stand to Sit and Talk", "Sit to Stand and Talk", "Stand to Walk", "Walk to Stand", "Stand to Climb Stairs / Stand to Climb Stairs and Talk", "Climb Stairs to Walk", "Climb Stairs and Talk to Walk and Talk"]

    X = data.drop(columns = ["Atividade"])
    y = np.array(list(map(lambda x: activities_labels.index(x),data["Atividade"])))
    
    for col in X.columns:
        col_data = X[col]
        minVal = min(col_data);
        maxVal = max(col_data);
        X[col] = (col_data - minVal) / ( maxVal - minVal )
        
    fisher_score = chi2(X, y)
    p_values = pd.Series(fisher_score[1], index = X.columns)
    p_values.sort_values(ascending = True, inplace = True)
    return p_values[:number_features]

In [8]:
print("Fisher Score top 10 features:")
display(list(fisher_score(all_features,10).index))
print("\n\nReliefF top 10 features:")
display(reliefF(all_features,10))

Fisher Score top 10 features:


['GYRO_Z_std',
 'MAG_X_DF',
 'MAG_X_spectral_entropy',
 'GYRO_Z_IQR',
 'MAG_Z_DF',
 'MAG_X_calculate_entropy',
 'MAG_Z_spectral_entropy',
 'MAG_X_std',
 'MAG_Z_calculate_entropy',
 'MAG_Y_DF']



ReliefF top 10 features:
Sucess!


['MAG_X_IQR',
 'MAG_Z_IQR',
 'MAG_Y_IQR',
 'MAG_Z_median',
 'MAG_X_median',
 'MAG_Y_median',
 'GYRO_Y_median',
 'ACC_X_IQR',
 'ACC_X_median',
 'ACC_Y_median']

## Conclusões

Após implementar os algoritmos de ReliefF e de Fisher Score para retirar as 10 melhores features para representar o dataset, podemos concluir que não há uma unica feature em comum entre as duas abordagens (como podemos ver em cima).

Respondendo ao 4.6.1, para obter as features relativas a esta compressão seria apenas necessário mapear o index desta tabela para a tabela com todas as features e dessa tabela com todas as features, podemos mapear para a janela de 2 segundos que tem o sinal RAW. 
O exemplo está no 4.4.1 onde fazemos o mesmo para o PCA.

A vantagem destas abordagens é que conseguimos reduzir bastante o numero de features sem perder a interpretabilidade dos dados como acontece no algoritmo do PCA. Outra vantagem é que assim podemos estar a retirar features que não teriam peso nenhum no machine learning, e deste modo, otimizamos os nossos algorimos de IA.

As limitações destas abordagens é que por exemplo neste caso concreto, 10 features pode não ser suficiente para representar o Dataset, estando assim a perder informação perciosa para o machine learning.
O que pode resultar em acabar em trial and error para encontrar o numero ideal de feature.